### Modelado

In [1]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install skfeature-chappers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression 

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
import multiprocessing  

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor
import numpy as np

import statsmodels as sm
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', None)

In [2]:
X_train = pd.read_csv('X_train.csv.gz')
X_test = pd.read_csv('X_test.csv.gz')
y_train = pd.read_csv('y_train.csv.gz')
y_test = pd.read_csv('y_test.csv.gz')

In [3]:
y_train = y_train.drop(columns=['price'])
y_test = y_test.drop(columns=['price'])

In [4]:
#selected_columns = ['accommodates','bedrooms', 'bathrooms_count' ,'number_of_reviews',
                              #'availability_30', 
                              #'antiguedad', 'host_is_superhost_encoded','room_type_encoded', 'amenity_count', 'neighbourhood_cleansed_encoded', 'month']

X_train_reg =  X_train.drop(['date', 'latitude', 'longitude', 'available', 'host_is_superhost', 'amenities', 'neighbourhood_cleansed', 'room_type'], axis=1)
X_test_reg = X_test.drop(['date', 'latitude', 'longitude', 'available', 'host_is_superhost', 'amenities', 'neighbourhood_cleansed', 'room_type'], axis=1)

#### Gradient Boosting 

In [6]:
boosting_model = GradientBoostingRegressor(random_state=42)

boosting_model.fit(X_train_rf, y_train.values.ravel())

y_train_pred_boosting = boosting_model.predict(X_train_rf)

y_test_pred_boosting = boosting_model.predict(X_test_rf)

r2_train_boosting = r2_score(y_train, y_train_pred_boosting)
mse_train_boosting = mean_squared_error(y_train, y_train_pred_boosting)

r2_test_boosting = r2_score(y_test, y_test_pred_boosting)
mse_test_boosting = mean_squared_error(y_test, y_test_pred_boosting)

In [7]:
print('Gradient Boosting Metrics:')
print('Training Set - R-squared:', r2_train_boosting)
print('Training Set - Mean Squared Error:', mse_train_boosting)
print('Test Set - R-squared:', r2_test_boosting)
print('Test Set - Mean Squared Error:', mse_test_boosting)

Gradient Boosting Metrics:
Training Set - R-squared: 0.5206425399757273
Training Set - Mean Squared Error: 0.4254067771219621
Test Set - R-squared: 0.5212639044453273
Test Set - Mean Squared Error: 0.4236505916240194


#### XGBoost (Sin GridSearch) 

In [8]:
xgb_model = xgb.XGBRegressor(random_state=42)

xgb_model.fit(X_train_rf, y_train.values.ravel())

y_train_pred_xgb = xgb_model.predict(X_train_rf)

y_test_pred_xgb = xgb_model.predict(X_test_rf)

r2_train_xgb = r2_score(y_train, y_train_pred_xgb)
mse_train_xgb = mean_squared_error(y_train, y_train_pred_xgb)

r2_test_xgb = r2_score(y_test, y_test_pred_xgb)
mse_test_xgb = mean_squared_error(y_test, y_test_pred_xgb)

In [9]:
print('XGBoost Metrics:')
print('Training Set - R-squared:', r2_train_xgb)
print('Training Set - Mean Squared Error:', mse_train_xgb)
print('Test Set - R-squared:', r2_test_xgb)
print('Test Set - Mean Squared Error:', mse_test_xgb)

XGBoost Metrics:
Training Set - R-squared: 0.7074762175432394
Training Set - Mean Squared Error: 0.25960084050878285
Test Set - R-squared: 0.7006104916257658
Test Set - Mean Squared Error: 0.26494042025766495


#### XGBoost (Con GridSearch)

In [16]:
xgb_model = xgb.XGBRegressor()

param_grid = {
    'max_depth': [3, 5, 7],  
    'learning_rate': [0.1, 0.01],  
    'n_estimators': [100, 200],  
    'min_child_weight': [1, 3],  
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_rf, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_estimator = grid_search.best_estimator_

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 200}


In [17]:
y_train_pred = best_estimator.predict(X_train_rf)

y_test_pred = best_estimator.predict(X_test_rf)

train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_r2 = r2_score(y_train, y_train_pred)

test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_r2 = r2_score(y_test, y_test_pred)

print("Training set metrics:")
print("MAE:", train_mae)
print("MSE:", train_mse)
print("RMSE:", train_rmse)
print("R-squared:", train_r2)
print()
print("Test set metrics:")
print("MAE:", test_mae)
print("MSE:", test_mse)
print("RMSE:", test_rmse)
print("R-squared:", test_r2)

Training set metrics:
MAE: 0.36178843813937644
MSE: 0.24324954261887066
RMSE: 0.4932033481423972
R-squared: 0.7259012099179176

Test set metrics:
MAE: 0.36436851581485924
MSE: 0.24750896793365998
RMSE: 0.497502731584119
R-squared: 0.7203084823531047


#### Adaboost 

In [10]:
adaboost_model = AdaBoostRegressor(random_state=42)

adaboost_model.fit(X_train_rf, y_train.values.ravel())

y_train_pred_adaboost = adaboost_model.predict(X_train_rf)

y_test_pred_adaboost = adaboost_model.predict(X_test_rf)

r2_train_adaboost = r2_score(y_train, y_train_pred_adaboost)
mse_train_adaboost = mean_squared_error(y_train, y_train_pred_adaboost)

r2_test_adaboost = r2_score(y_test, y_test_pred_adaboost)
mse_test_adaboost = mean_squared_error(y_test, y_test_pred_adaboost)

In [11]:
print('AdaBoost Metrics:')
print('Training Set - R-squared:', r2_train_adaboost)
print('Training Set - Mean Squared Error:', mse_train_adaboost)
print('Test Set - R-squared:', r2_test_adaboost)
print('Test Set - Mean Squared Error:', mse_test_adaboost)

AdaBoost Metrics:
Training Set - R-squared: 0.3188146336095182
Training Set - Mean Squared Error: 0.6045193733381024
Test Set - R-squared: 0.3186852322952404
Test Set - Mean Squared Error: 0.6029196609582566
